In [ ]:
#not working part1
import sys
import re
from collections import deque

def parse_input(lines):
    # Split into two sections: shapes then regions
    lines = [ln.rstrip('\n') for ln in lines]
    sep = None
    for i,ln in enumerate(lines):
        if re.match(r'\d+x\d+\s*:', ln):
            sep = i-1
            break
    if sep is None:
        raise ValueError("Can't find separator between shapes and regions")
    print(f'sep {sep}')
    if sep is None:
        shape_lines = lines
        reg_lines = []
    else:
        shape_lines = lines[:sep]
        reg_lines = lines[sep+1:]
    print(f'shape_lines {shape_lines} reg_lines {reg_lines}')

    #parse shapes
    shapes = {}
    i = 0
    while i < len(shape_lines):
        ln = shape_lines[i].strip()
        if not ln:
            i += 1
            continue
        m = re.match(r'(\d+)\s*:', ln)
        if not m:
            raise ValueError(f"Expected shape index at line: {shape_lines[i]!r}")
        idx = int(m.group(1))
        i += 1
        rows = []
        # lines of '.' and '#'
        while i < len(shape_lines) and re.match(r'^[.#]+$', shape_lines[i].strip()):
            rows.append(shape_lines[i].strip())
            i += 1
        if not rows:
            raise ValueError(f"No rows found for shape {idx}")
        shapes[idx] = rows

    print(f'shapes len(shape_liens) {len(shape_lines)}')
    print(shapes)

    # parse regions
    regions = []
    for ln in reg_lines:
        ln = ln.strip()
        if not ln:
            continue
        m = re.match(r'(\d+)x(\d+)\s*:\s*(.*)$', ln)
        print(f'm {m} ln {ln}')
        if not m:
            raise ValueError(f"Bad region line: {ln}")
        w = int(m.group(1))
        h = int(m.group(2))
        counts = [int(x) for x in m.group(3).split()]
        regions.append((w, h, counts))
    return shapes, regions

# Helpers to convert shape strings to set of coordinates and normalize
def shape_to_cells(rows):
    cells = set()
    for y, row in enumerate(rows):
        for x, ch in enumerate(row):
            if ch == '#':
                cells.add((x, y))
    return normalize(cells)

def normalize(cells):
    if not cells:
        return frozenset()
    minx = min(x for x,y in cells)
    miny = min(y for x,y in cells)
    norm = frozenset(((x-minx, y-miny) for x,y in cells))
    return norm

# generate all distinct rotations and flips
def generate_orientations(cells):
    orientations = set()
    def rot90(cset):
        return set(( -y, x) for x,y in cset)
    def flipx(cset):
        return set(( -x, y) for x,y in cset)
    cur = set(cells)
    for r in range(4):
        # normalized rotations
        norm = normalize(cur)
        orientations.add(norm)
        # flip
        fx = normalize(flipx(cur))
        orientations.add(fx)
        # rotate 90
        cur = rot90(cur)
    result = [set(o) for o in orientations]
    return result

# Check if an orientation fits at position
def can_place(orient, ox, oy, W, H, occ):
    for dx,dy in orient:
        x = ox + dx
        y = oy + dy
        if x < 0 or x >= W or y < 0 or y >= H:
            return False
        if occ[y][x]:
            return False
    return True

def place(orient, ox, oy, occ, val=True):
    for dx,dy in orient:
        occ[oy+dy][ox+dx] = val

# flood fill to find connected free region sizes
def free_region_sizes(occ, W, H):
    seen = [[False]*W for _ in range(H)]
    regions = []
    for y in range(H):
        for x in range(W):
            if not occ[y][x] and not seen[y][x]:
                # BFS
                q = deque([(x,y)])
                seen[y][x] = True
                cnt = 0
                while q:
                    cx,cy = q.popleft()
                    cnt += 1
                    for dx,dy in ((1,0),(-1,0),(0,1),(0,-1)):
                        nx, ny = cx+dx, cy+dy
                        if 0 <= nx < W and 0 <= ny < H and not occ[ny][nx] and not seen[ny][nx]:
                            seen[ny][nx] = True
                            q.append((nx,ny))
                regions.append(cnt)
    return regions

# Backtracking placement
def can_pack(W, H, piece_list, orientations_by_shape):
    # piece_list: list of shape indices (one entry per required present)
    # orientations_by_shape: dict shape_idx -> list of orientations (each is set of (dx,dy))

    # Precompute shape areas
    areas = {s: next(iter(orientations_by_shape[s])).__len__() if False else None for s in orientations_by_shape}
    # Compute areas
    for s in orientations_by_shape:
        if orientations_by_shape[s]:
            areas[s] = len(next(iter(orientations_by_shape[s])))
        else:
            areas[s] = 0

    piece_list = sorted(piece_list, key=lambda si: -areas[si])

    occ = [[False]*W for _ in range(H)]
    total_needed_area = sum(areas[s] for s in piece_list)

    if total_needed_area > W*H:
        return False

    # Pre-generate placements
    placements_by_shape = {}
    for s in orientations_by_shape:
        placements = []  # each: (orient, ox, oy)
        for orient in orientations_by_shape[s]:
            maxx = max(x for x,y in orient)
            maxy = max(y for x,y in orient)
            for oy in range(0, H - maxy):
                for ox in range(0, W - maxx):
                    placements.append((orient, ox, oy))
        placements_by_shape[s] = placements

    # recursive search
    seen_states = 0
    def backtrack(idx, remaining_area):
        nonlocal seen_states
        seen_states += 1
        if idx >= len(piece_list):
            return True
        # pruning: area left vs free cells
        free_cells = sum(1 for y in range(H) for x in range(W) if not occ[y][x])
        if remaining_area > free_cells:
            return False
        # connected-region pruning: smallest remaining piece area
        rem_areas = [areas[p] for p in piece_list[idx:]]
        min_area = min(rem_areas) if rem_areas else 0
        regions = free_region_sizes(occ, W, H)
        for rsz in regions:
            if rsz < min_area:
                return False

        shape = piece_list[idx]
        # try all placements for this shape
        for orient, ox, oy in placements_by_shape[shape]:
            if not can_place(orient, ox, oy, W, H, occ):
                continue
            place(orient, ox, oy, occ, True)
            ok = backtrack(idx+1, remaining_area - areas[shape])
            if ok:
                return True
            place(orient, ox, oy, occ, False)
        return False

    return backtrack(0, total_needed_area)

def solve_from_text(text):
    shapes, regions = parse_input(text.splitlines())
    orientations_by_shape = {}
    for sidx, rows in shapes.items():
        cells = shape_to_cells(rows)
        orients = generate_orientations(cells)
        norm_orients = []
        for o in orients:
            norm_orients.append(set(o))
        # dedupe
        uniq = []
        seen = set()
        for o in norm_orients:
            key = tuple(sorted(o))
            if key not in seen:
                seen.add(key)
                uniq.append(o)
        orientations_by_shape[sidx] = uniq

    # For each region compute whether packable
    results = []
    for (W, H, counts) in regions:
        piece_list = []
        for i, c in enumerate(counts):
            if c > 0:
                if i not in orientations_by_shape:
                    raise ValueError(f"Region requests shape {i} but shape not provided")
                piece_list.extend([i]*c)
        ok = can_pack(W, H, piece_list, orientations_by_shape)
        results.append(((W,H,counts), ok))
    return results

if __name__ == "__main__":
    with open('day12.txt', 'r') as f:
        text = f.read()
    # print(text)

    res = solve_from_text(text)
    count_ok = sum(1 for _,ok in res if ok)
    for (W,H,counts), ok in res:
        print(f"{W}x{H}: {'OK' if ok else 'NO'}  counts={counts}")
    print()
    print(f"Regions that can fit all presents: {count_ok} / {len(res)}")


sep 29
shape_lines ['0:', '###', '##.', '##.', '', '1:', '###', '##.', '.##', '', '2:', '.##', '###', '##.', '', '3:', '##.', '###', '##.', '', '4:', '###', '#..', '###', '', '5:', '###', '.#.', '###'] reg_lines ['4x4: 0 0 0 0 2 0', '12x5: 1 0 1 0 2 2']
shapes len(shape_liens) 29
{0: ['###', '##.', '##.'], 1: ['###', '##.', '.##'], 2: ['.##', '###', '##.'], 3: ['##.', '###', '##.'], 4: ['###', '#..', '###'], 5: ['###', '.#.', '###']}
m <re.Match object; span=(0, 16), match='4x4: 0 0 0 0 2 0'> ln 4x4: 0 0 0 0 2 0
m <re.Match object; span=(0, 17), match='12x5: 1 0 1 0 2 2'> ln 12x5: 1 0 1 0 2 2
4x4: OK  counts=[0, 0, 0, 0, 2, 0]
12x5: NO  counts=[1, 0, 1, 0, 2, 2]

Regions that can fit all presents: 1 / 2


In [ ]:
#works part1 but only on smaller regions
import sys
import re

# ----------------------------------------------------------------------
# Parsing
def parse_input(lines):
    # Split into two sections: shapes then regions
    lines = [ln.rstrip('\n') for ln in lines]
    sep = None
    for i,ln in enumerate(lines):
        if re.match(r'\d+x\d+\s*:', ln):
            sep = i-1
            break
    if sep is None:
        raise ValueError("Can't find separator between shapes and regions")
    print(f'sep {sep}')
    if sep is None:
        shape_lines = lines
        reg_lines = []
    else:
        shape_lines = lines[:sep]
        reg_lines = lines[sep+1:]
    print(f'shape_lines {shape_lines} reg_lines {reg_lines}')

    # parse shapes
    shapes = {}
    i = 0
    while i < len(shape_lines):
        ln = shape_lines[i].strip()
        if not ln:
            i += 1
            continue
        m = re.match(r'(\d+)\s*:', ln)
        if not m:
            raise ValueError(f"Expected shape index at line: {shape_lines[i]!r}")
        idx = int(m.group(1))
        i += 1
        rows = []
        # collect lines that look like '.' and '#'
        while i < len(shape_lines) and re.match(r'^[.#]+$', shape_lines[i].strip()):
            rows.append(shape_lines[i].strip())
            i += 1
        if not rows:
            raise ValueError(f"No rows found for shape {idx}")
        shapes[idx] = rows

    print(f'shapes len(shape_liens) {len(shape_lines)}')
    print(shapes)

    # parse regions: lines like "4x4: 0 0 0 0 2 0"
    regions = []
    for ln in reg_lines:
        ln = ln.strip()
        if not ln:
            continue
        m = re.match(r'(\d+)x(\d+)\s*:\s*(.*)$', ln)
        print(f'm {m} ln {ln}')
        if not m:
            raise ValueError(f"Bad region line: {ln}")
        w = int(m.group(1))
        h = int(m.group(2))
        counts = [int(x) for x in m.group(3).split()]
        regions.append((w, h, counts))
    return shapes, regions

# ----------------------------------------------------------------------
# Shape generation (rotations + flips)

def normalize(cells):
    minx = min(x for x, y in cells)
    miny = min(y for x, y in cells)
    return frozenset(((x - minx, y - miny) for x, y in cells))

def shape_to_cells(rows):
    cells = []
    for y, row in enumerate(rows):
        for x, ch in enumerate(row):
            if ch == "#":
                cells.append((x, y))
    return normalize(cells)

def rot90(cells):
    return {(-y, x) for x, y in cells}

def flipx(cells):
    return {(-x, y) for x, y in cells}

def generate_orientations(cells):
    out = set()
    cur = cells
    for _ in range(4):
        out.add(normalize(cur))
        out.add(normalize(flipx(cur)))
        cur = rot90(cur)
    # return sorted list of sorted lists (stable)
    return [sorted(o) for o in out]


# ----------------------------------------------------------------------
# DLX implementation with secondary columns
class DLXNode:
    __slots__ = ("L","R","U","D","C","row_id")

class DLXColumn:
    __slots__ = ("name","size","L","R","U","D","primary")

def build_dlx(primary_cols, secondary_cols, rows):
    header = DLXColumn()
    header.L = header
    header.R = header

    col_lookup = {}

    # Add primary columns to linked list
    last = header
    for name in primary_cols:
        col = DLXColumn()
        col.name = name
        col.size = 0
        col.U = col
        col.D = col
        col.primary = True
        col_lookup[name] = col

        col.L = last
        col.R = last.R
        last.R.L = col
        last.R = col
        last = col

    # Secondary columns (not linked left/right)
    for name in secondary_cols:
        col = DLXColumn()
        col.name = name
        col.size = 0
        col.U = col
        col.D = col
        col.primary = False
        col_lookup[name] = col

    # Build row nodes
    for rid, cols in rows.items():
        first = None
        for cname in cols:
            col = col_lookup[cname]
            node = DLXNode()
            node.C = col
            node.row_id = rid

            # vertical insert
            node.U = col.U
            node.D = col
            col.U.D = node
            col.U = node
            col.size += 1

            # horizontal row insert
            if first is None:
                first = node
                node.L = node
                node.R = node
            else:
                node.L = first.L
                node.R = first
                first.L.R = node
                first.L = node

    return header

def cover(col):
    if col.primary:
        col.R.L = col.L
        col.L.R = col.R

    row = col.D
    while row != col:
        j = row.R
        while j != row:
            j.D.U = j.U
            j.U.D = j.D
            j.C.size -= 1
            j = j.R
        row = row.D


def uncover(col):
    row = col.U
    while row != col:
        j = row.L
        while j != row:
            j.C.size += 1
            j.D.U = j
            j.U.D = j
            j = j.L
        row = row.U

    if col.primary:
        col.R.L = col
        col.L.R = col


def solve_dlx(header, solutions, limit=1):
    # If no primary columns remain → success
    if header.R == header:
        solutions.append(True)
        return True

    # choose smallest primary column
    c = header.R
    best = c
    min_sz = c.size
    while c != header:
        if c.size < min_sz:
            min_sz = c.size
            best = c
        c = c.R
    c = best

    if min_sz == 0:
        return False

    cover(c)

    r = c.D
    while r != c:
        j = r.R
        while j != r:
            cover(j.C)
            j = j.R

        if solve_dlx(header, solutions, limit):
            return True

        j = r.L
        while j != r:
            uncover(j.C)
            j = j.L

        r = r.D

    uncover(c)
    return False


# ----------------------------------------------------------------------
# Region packing with DLX
def can_pack_region(W, H, piece_counts, orientations_by_shape):

    primary_cols = []
    secondary_cols = []

    # Present-instance columns (must be filled)
    for shape_idx, count in enumerate(piece_counts):
        for k in range(count):
            primary_cols.append(f"P-{shape_idx}-{k}")

    # Cell columns (optional – at most once)
    for y in range(H):
        for x in range(W):
            secondary_cols.append(f"C-{x}-{y}")

    rows = {}
    row_id = 0

    for shape_idx, count in enumerate(piece_counts):
        if count == 0:
            continue
        orients = orientations_by_shape[shape_idx]

        for inst in range(count):
            pcol = f"P-{shape_idx}-{inst}"

            for orient in orients:
                maxx = max(x for x, y in orient)
                maxy = max(y for x, y in orient)

                for oy in range(H - maxy):
                    for ox in range(W - maxx):

                        used = [pcol]
                        for dx, dy in orient:
                            used.append(f"C-{ox+dx}-{oy+dy}")

                        rows[row_id] = used
                        row_id += 1

    header = build_dlx(primary_cols, secondary_cols, rows)

    out = []
    ok = solve_dlx(header, out, limit=1)
    return ok


# ----------------------------------------------------------------------
# Solve
def solve_text(text):
    shapes, regions = parse_input(text.splitlines())

    orientations_by_shape = {
        sid: generate_orientations(shape_to_cells(rows))
        for sid, rows in shapes.items()
    }

    results = []
    for (W, H, counts) in regions:
        ok = can_pack_region(W, H, counts, orientations_by_shape)
        results.append(((W, H, counts), ok))
    return results


# ----------------------------------------------------------------------
# main run

if __name__ == "__main__":
    with open('day12.txt', 'r') as f:
        text = f.read()
    results = solve_text(text)

    total = sum(1 for _, ok in results if ok)
    print(f'total {total}')
    for (W, H, c), ok in results:
        print(f"{W}x{H}  {c}  →  {'OK' if ok else 'NO'}")
    print(f"\nFillable regions: {total} / {len(results)}")


sep 29
shape_lines ['0:', '###', '##.', '##.', '', '1:', '###', '##.', '.##', '', '2:', '.##', '###', '##.', '', '3:', '##.', '###', '##.', '', '4:', '###', '#..', '###', '', '5:', '###', '.#.', '###'] reg_lines ['4x4: 0 0 0 0 2 0', '12x5: 1 0 1 0 2 2', '12x5: 1 0 1 0 3 2']
shapes len(shape_liens) 29
{0: ['###', '##.', '##.'], 1: ['###', '##.', '.##'], 2: ['.##', '###', '##.'], 3: ['##.', '###', '##.'], 4: ['###', '#..', '###'], 5: ['###', '.#.', '###']}
m <re.Match object; span=(0, 16), match='4x4: 0 0 0 0 2 0'> ln 4x4: 0 0 0 0 2 0
m <re.Match object; span=(0, 17), match='12x5: 1 0 1 0 2 2'> ln 12x5: 1 0 1 0 2 2
m <re.Match object; span=(0, 17), match='12x5: 1 0 1 0 3 2'> ln 12x5: 1 0 1 0 3 2
total 2
4x4  [0, 0, 0, 0, 2, 0]  →  OK
12x5  [1, 0, 1, 0, 2, 2]  →  OK
12x5  [1, 0, 1, 0, 3, 2]  →  NO

Fillable regions: 2 / 3


In [ ]:
#part1 works on smaller regions but not larger regions

import numpy as np
from functools import lru_cache
import random
import sys
sys.setrecursionlimit(10_000_000)

# ------------------------------------------------------------
# ZOBRIST HASH INITIALIZATION

def make_zobrist(width, height):
    table = np.random.randint(0, 2**63-1, size=(height, width), dtype=np.uint64)
    return table

# ------------------------------------------------------------
# SHAPE NORMALIZATION

def parse_shape(lines):
    coords = []
    for y, row in enumerate(lines):
        for x, ch in enumerate(row):
            if ch == '#':
                coords.append((x, y))
    return normalize(coords)

def normalize(coords):
    minx = min(x for x, y in coords)
    miny = min(y for x, y in coords)
    return sorted([(x-minx, y-miny) for x, y in coords])

def rotate(coords):
    return normalize([(-y, x) for (x, y) in coords])

def reflect(coords):
    return normalize([(-x, y) for (x, y) in coords])

def all_orientations(coords):
    seen = set()
    out = []
    c = coords
    for _ in range(4):
        c = rotate(c)
        for r in (c, reflect(c)):
            t = tuple(r)
            if t not in seen:
                seen.add(t)
                out.append(r)
    return out

# ------------------------------------------------------------
# BITBOARD

def coords_to_bitmask(coords, W, H):
    """Return uint64 mask with bits for each shape tile."""
    mask = 0
    for (x, y) in coords:
        idx = y * W + x
        mask |= (1 << idx)
    return mask

def place_shape_on_board(shape_coords, W, H):
    """Generate all valid placements of a shape, each as a uint64 bitmask."""
    placed = []
    maxx = max(x for x, y in shape_coords)
    maxy = max(y for x, y in shape_coords)

    for oy in range(H - maxy):
        for ox in range(W - maxx):
            mask = 0
            ok = True
            for (sx, sy) in shape_coords:
                x = ox + sx
                y = oy + sy
                idx = y * W + x
                mask |= (1 << idx)
            placed.append(mask)
    return placed

# ------------------------------------------------------------
# DLX EXACT COVER USING BITMASKS (FAST BRANCH & BOUND)

def solve_region(W, H, shapes, shape_counts):

    total_cells = W * H
    zob = make_zobrist(W, H)

    # Precompute all placements
    all_placements = []
    for i, base_coords in enumerate(shapes):
        orientations = all_orientations(base_coords)
        per_shape = []
        for orient in orientations:
            per_shape.extend(place_shape_on_board(orient, W, H))
        all_placements.append(per_shape)

    # Convert shape index with count into a flat list, e.g. [0,0,2,5,5,5]
    needed = []
    for i, c in enumerate(shape_counts):
        needed.extend([i] * c)

    needed = tuple(needed)
    N = total_cells

    @lru_cache(None)
    def can_fill(idx, occ_mask):
        """idx = which shape in 'needed' list to place next."""
        if idx == len(needed):
            return True

        shape_id = needed[idx]

        # Try each placement of that shape
        for pmask in all_placements[shape_id]:

            # Check if pmask conflicts
            if pmask & occ_mask:
                continue

            new_occ = occ_mask | pmask

            if can_fill(idx+1, new_occ):
                return True

        return False

    return can_fill(0, 0)

# ------------------------------------------------------------
# INPUT PARSING

def parse_input(raw_text):
    lines = [l.rstrip() for l in raw_text.splitlines() if l.strip()]

    # First parse shapes
    i = 0
    shapes = []
    while i < len(lines):
        if lines[i][0].isdigit() and lines[i].endswith(':'):
            idx = int(lines[i][:-1])
            i += 1
            shape_rows = []
            while i < len(lines) and not (lines[i][0].isdigit() and lines[i].endswith(':')) and 'x' not in lines[i]:
                shape_rows.append(lines[i])
                i += 1
            shapes.append(parse_shape(shape_rows))
        else:
            break

    # Next parse regions
    regions = []
    while i < len(lines):
        line = lines[i]
        if 'x' in line:
            dim, rest = line.split(':')
            W, H = map(int, dim.split('x'))
            counts = list(map(int, rest.strip().split()))
            regions.append((W, H, counts))
        i += 1

    return shapes, regions

# ------------------------------------------------------------
# main

if __name__ == "__main__":
    with open('day12.txt', 'r') as f:
        text = f.read()
    shapes, regions = parse_input(text)

    count = 0
    for (W, H, shape_counts) in regions:
        ok = solve_region(W, H, shapes, shape_counts)
        print(f"{W}x{H} -> {'OK' if ok else 'FAIL'}")
        if ok:
            count += 1

    print("Total regions solvable:", count)


4x4 -> OK
12x5 -> OK
12x5 -> FAIL
Total regions solvable: 2


In [ ]:
#part 1. Doesn't work on smaller grids. Works on input/large regions
import re
def parse_input(lines):
    # Split into two sections: shapes (until first blank line), then regions
    lines = [ln.rstrip('\n') for ln in lines]
    sep = None
    for i,ln in enumerate(lines):
        if re.match(r'\d+x\d+\s*:', ln):
            sep = i-1
            break
    if sep is None:
        raise ValueError("Can't find separator between shapes and regions")
    if sep is None:
        shape_lines = lines
        reg_lines = []
    else:
        shape_lines = lines[:sep]
        reg_lines = lines[sep+1:]

    # parse shapes
    shapes = {}
    i = 0
    while i < len(shape_lines):
        ln = shape_lines[i].strip()
        if not ln:
            i += 1
            continue
        m = re.match(r'(\d+)\s*:', ln)
        if not m:
            raise ValueError(f"Expected shape index at line: {shape_lines[i]!r}")
        idx = int(m.group(1))
        i += 1
        rows = []
        # parse lines '.' and '#'
        while i < len(shape_lines) and re.match(r'^[.#]+$', shape_lines[i].strip()):
            rows.append(shape_lines[i].strip())
            i += 1
        if not rows:
            raise ValueError(f"No rows found for shape {idx}")
        shapes[idx] = rows

    # parse regions "4x4: 0 0 0 0 2 0"
    regions = []
    for ln in reg_lines:
        ln = ln.strip()
        if not ln:
            continue
        m = re.match(r'(\d+)x(\d+)\s*:\s*(.*)$', ln)
        if not m:
            raise ValueError(f"Bad region line: {ln}")
        w = int(m.group(1))
        h = int(m.group(2))
        counts = [int(x) for x in m.group(3).split()]
        regions.append((w, h, counts))
    return shapes, regions

with open('input.txt', 'r') as f:
    text = f.read()
shapes, regions_input = parse_input(text.splitlines())
raw_shapes=[v for i,v in sorted(shapes.items())]



from itertools import product

# -----------------------------
# 1. Define the present shapes

# raw_shapes = [['###', '##.', '##.'],
#  ['###', '##.', '.##'],
#  ['.##', '###', '##.'],
#  ['##.', '###', '##.'],
#  ['###', '#..', '###'],
#  ['###', '.#.', '###']]

# -----------------------------
# 2. Convert shapes to bitmasks
def shape_to_mask(shape):
    h, w = len(shape), len(shape[0])
    mask = 0
    for r in range(h):
        for c in range(w):
            if shape[r][c] == '#':
                mask |= 1 << (r * w + c)
    return mask, h, w, sum(row.count('#') for row in shape)

# Generate all rotations and flips
def generate_variants(shape):
    variants = set()
    grid = [list(row) for row in shape]
    for _ in range(2):  # flip
        for _ in range(4):  # rotate
            h, w = len(grid), len(grid[0])
            bm, _, _, hashes = shape_to_mask([''.join(row) for row in grid])
            variants.add((bm, h, w, hashes))
            # rotate 90
            grid = [ [grid[h-1-r][c] for r in range(h)] for c in range(w)]
        # horizontal flip
        grid = [row[::-1] for row in grid]
    return list(variants)

present_variants = [generate_variants(shape) for shape in raw_shapes]

# -----------------------------
# 3. Helper functions
def can_place(grid, bm, r0, c0, h, w):
    for r in range(h):
        row_bits = (bm >> (r*w)) & ((1<<w)-1)
        if (grid[r0+r] >> c0) & row_bits:
            return False
    return True

def place_tile(grid, bm, r0, c0, h, w):
    for r in range(h):
        row_bits = (bm >> (r*w)) & ((1<<w)-1)
        grid[r0+r] |= row_bits << c0

def remove_tile(grid, bm, r0, c0, h, w):
    for r in range(h):
        row_bits = (bm >> (r*w)) & ((1<<w)-1)
        grid[r0+r] ^= row_bits << c0

# -----------------------------
# 4. Optimized backtracking
def solve_region(grid, presents_to_place, total_hashes_left, grid_width):
    if not presents_to_place:
        return True
    # pruning
    empty_cells = sum(bin((1<<grid_width)-1 ^ row).count('1') for row in grid)
    if total_hashes_left > empty_cells:
        return False

    # greedy: place largest present first
    presents_to_place.sort(key=lambda x: -x[3])
    tile_id, _, _, tile_hashes = presents_to_place[0]

    for bm, h, w, _ in present_variants[tile_id]:
        for r in range(len(grid)-h+1):
            for c in range(grid_width-w+1):
                if can_place(grid, bm, r, c, h, w):
                    place_tile(grid, bm, r, c, h, w)
                    if solve_region(grid, presents_to_place[1:], total_hashes_left - tile_hashes, grid_width):
                        return True
                    remove_tile(grid, bm, r, c, h, w)
    return False

# -----------------------------
# 5. Input regions and counts

# regions_input = [(4, 4, [0, 0, 0, 0, 2, 0]),
#  (12, 5, [1, 0, 1, 0, 2, 2]),
#  (12, 5, [1, 0, 1, 0, 3, 2])]

# -----------------------------
# 6. main

fillable_count = 0

for w,h,counts in regions_input:
    print(f'{w}x{h}')
    total_hashes = sum(counts[i]*present_variants[i][0][3] for i in range(len(counts)))
    presents_to_place = []
    for idx, qty in enumerate(counts):
        for _ in range(qty):
            presents_to_place.append([idx,0,0,present_variants[idx][0][3]])  # only need tile_id and hash count
    grid = [0]*h
    if solve_region(grid, presents_to_place, total_hashes, w):
        fillable_count += 1

print("Regions that can fit all presents:", fillable_count)


['0:', '.##', '##.', '###', '', '1:', '..#', '.##', '##.', '', '2:', '###', '.##', '..#', '', '3:', '###', '..#', '###', '', '4:', '###', '###', '..#', '', '5:', '###', '.#.', '###', '', '47x48: 59 59 54 61 53 61', '40x36: 17 35 30 26 24 23', '38x41: 36 44 38 37 38 48', '45x44: 28 51 42 26 31 32', '47x44: 60 41 68 54 52 42', '36x42: 39 37 49 38 35 36', '47x40: 45 49 49 53 47 47', '44x44: 35 57 68 51 45 47', '38x47: 38 54 57 38 46 46', '37x41: 31 22 22 22 26 32', '37x46: 34 26 24 32 27 36', '44x43: 53 43 50 52 43 49', '40x42: 40 32 20 28 33 29', '41x38: 37 42 43 38 33 48', '35x36: 17 33 22 12 26 22', '50x50: 63 63 61 63 65 69', '41x50: 63 47 60 36 47 62', '37x43: 47 37 28 37 44 49', '41x36: 42 40 37 36 38 35', '44x38: 42 54 46 41 42 36', '40x46: 56 54 45 37 50 43', '41x50: 40 34 39 26 36 32', '35x37: 26 35 39 39 28 34', '35x41: 24 23 19 23 21 32', '41x40: 40 38 46 55 36 37', '48x41: 39 38 24 31 39 37', '45x43: 42 32 33 31 38 34', '40x43: 49 35 42 41 51 44', '39x38: 34 27 23 23 33 16', '